In [119]:
import akshare as ak
import json
import pandas as pd
import sys
import datetime
import os
import pandas as pd
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.faker import Faker
from pyecharts.commons.utils import JsCode

import matplotlib.pyplot as plt
%matplotlib inline


def convert_time(x):
    return str(x).replace(' 00:00:00','')
import re
sys.path.append('..')
from configure.settings import DBSelector
db = DBSelector()
mongo = db.mongo('qq')
fund = mongo['fund']

In [10]:
def get_net_value(code):
    fund_open_fund_info_em_df = ak.fund_open_fund_info_em(fund=code, indicator="单位净值走势")
    return fund_open_fund_info_em_df

In [44]:


def dump_mongo(df,code):
    js_str = df.to_json(orient='index')
    js_dict = json.loads(js_str)
    js_list = js_dict.values()
    try:
        fund[code].insert_many(js_list)
    except Exception as e:
        print(e)
        


In [5]:
code_list =['040046',
 '050025',
 '012348',
 '501311',
 '000071',
 '006327',
 '164906',
 '040018',
 '002685']

In [6]:
def start_crawl(code):
    df = get_net_value(code)
    df['净值日期']=df['净值日期'].astype(str)
    dump_mongo(df,code)


In [59]:
for code in code_list:
    start_crawl(code)

In [17]:
code_list = re.findall('\(F(\d+)\)',txt)

In [16]:
fund_name = re.findall('\$(.*?)\$',txt)

In [18]:
fund_dict = dict(zip(code_list,fund_name))

In [19]:
fund_dict

{'040046': '华安纳斯达克100指数(F040046)',
 '050025': '博时标普500ETF联接(F050025)',
 '012348': '天弘恒生科技指数A(F012348)',
 '501311': '嘉实港股通新经济指数A(F501311)',
 '000071': '华夏恒生ETF联接(F000071)',
 '006327': '易方达中概互联50ETF联接人民币A(F006327)',
 '164906': '交银中证海外中国互联网指数(F164906)',
 '040018': '华安香港精选(F040018)',
 '002685': '中欧丰泓沪港深A(F002685)'}

In [60]:
# after fetch data

In [23]:
def plot_profit_line(df,code):
    title="{}基金收益率曲线".format(code)
    X=df['净值日期'].tolist()
    Y=list(map(lambda x:round(x,2),df['单位净值'].tolist()))
    c = (
        Line()
        .add_xaxis(X)
        .add_yaxis('', Y, is_smooth=True,
            label_opts=opts.LabelOpts(is_show=False),
            linestyle_opts=opts.LineStyleOpts(width=1,color='rgb(255, 0, 0)'),
        ).set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            xaxis_opts=opts.AxisOpts(
                                    name='日期',
                                    # min_interval=5,
                                    splitline_opts=opts.SplitLineOpts(is_show=True),
                                            axislabel_opts=opts.LabelOpts(rotate=45),

                                    ),
            yaxis_opts=opts.AxisOpts(
                                    min_=min(Y),
                                    max_=max(Y),
                splitline_opts=opts.SplitLineOpts(is_show=True),
            )
                                        ).set_colors(['green'])
        .render(f"../plot_image/{title}.html")
    )

In [13]:
txt='''
1、$华安纳斯达克100指数(F040046)$ ：跟踪纳斯达克100指数，近一年回报6.14%，最大回撤16.14%，最新规模27.65亿。

2、$博时标普500ETF联接(F050025)$：本基金为标普500ETF的场外联接基金，跟踪标普500指数，近一年回报13.06%，最大回撤9.95%，最新规模18.67亿。

3、$天弘恒生科技指数A(F012348)$ ：跟踪恒生科技指数，成立以来回报-38.99%，最大回撤-30.59%，最新规模14.11亿。

4、$嘉实港股通新经济指数A(F501311)$ ：跟踪港股通新经济指数，近一年回报-34.42%，最大回撤-36.59%，最新规模14.11亿。

5、$华夏恒生ETF联接(F000071)$ ：本基金为恒生ETF的场外联接基金，跟踪香港恒生指数，近一年回报-16.51%，最大回撤-24.79%，最新规模31.09亿。

6、$易方达中概互联50ETF联接人民币A(F006327)$ ：本基金为中概互联ETF的场外联接基金，跟踪中证海外中国互联网50指数，，近一年回报-48.18%，最大回撤-53.35%，最新规模74.72亿。

7、$交银中证海外中国互联网指数(F164906)$ ：跟踪中证海外中国互联网指数，近一年回报-55.37%，最大回撤-60.78%，最新规模121.22亿。

8、$华安香港精选(F040018)$ ：双基金经理苏圻涵、翁启森共同管理11年，挖掘多只新能源翻倍股。近一年回报-15.84%，最大回撤-22.16%，最新规模6.47亿。

9、$中欧丰泓沪港深A(F002685)$ ：拥有10余年港股研究经验的罗佳明管理，熟悉香港市场的环境和规则。近一年回报-11.20%，最大回撤-18.17%，合并规模68.05亿

'''

In [24]:

for code,name in fund_dict.items():
    result = []
    for item in fund[code].find({},{'_id':0}):
        result.append(item)
    df = pd.DataFrame(result)
    plot_profit_line(df,name)
    

In [26]:
name_list  = list(fund_dict.values())

In [28]:
sipmle_name = [i.split('(')[0] for i in name_list]

In [29]:
sipmle_name

['华安纳斯达克100指数',
 '博时标普500ETF联接',
 '天弘恒生科技指数A',
 '嘉实港股通新经济指数A',
 '华夏恒生ETF联接',
 '易方达中概互联50ETF联接人民币A',
 '交银中证海外中国互联网指数',
 '华安香港精选',
 '中欧丰泓沪港深A']

In [31]:
W=10000
focus_num = [
2.7*W,
1.7*W,
1.2*W,
1.2*W,
2.9*W,
11.2*W,
9.3*W,
1569,
2191,
]

In [56]:
from pyecharts import options as opts
from pyecharts.charts import Scatter
from pyecharts.faker import Faker
from pyecharts.options import InitOpts

c = (
    Scatter(InitOpts(width='800px',height='500px'))
    .add_xaxis(sipmle_name)
    .add_yaxis('A',focus_num,label_opts=opts.LabelOpts(is_show=True))
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
                                    # name='日期',
                                    # min_interval=5,
                                    splitline_opts=opts.SplitLineOpts(is_show=True),
                                            axislabel_opts=opts.LabelOpts(rotate=-45),

                                    ),
        yaxis_opts=opts.AxisOpts(    
            min_=0,
                splitline_opts=opts.SplitLineOpts(is_show=True),
            ),
        title_opts=opts.TitleOpts(title="welcome"),
        visualmap_opts=opts.VisualMapOpts(type_="size",max_=150000, min_=2000),
    )
    .render("welcome_fund.html")
)

In [47]:
focus_num

[27000.0, 17000.0, 12000.0, 12000.0, 29000.0, 112000.0, 93000.0, 1569, 2191]

In [76]:
from pyecharts import options as opts
from pyecharts.charts import Bar, Grid, Line
from pyecharts.faker import Faker

In [96]:
grid = (
    Grid(init_opts=opts.InitOpts())
    .add(c, grid_opts=opts.GridOpts(pos_bottom="40%"))
    .render("grid_vertical.html")
)

In [95]:
from pyecharts import options as opts
from pyecharts.charts import Bar

c = (
    Bar()
    .add_xaxis(
    new_name
    )
    .add_yaxis("基金关注度", focus_num,label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=0)),
        title_opts=opts.TitleOpts(title="基金关注度"),
    )
    # .render("基金关注度.html")
)

In [72]:
new_name = []
for name in sipmle_name:
    new_name.append('\n'.join(list(name)))

In [73]:
new_name

['华\n安\n纳\n斯\n达\n克\n1\n0\n0\n指\n数',
 '博\n时\n标\n普\n5\n0\n0\nE\nT\nF\n联\n接',
 '天\n弘\n恒\n生\n科\n技\n指\n数\nA',
 '嘉\n实\n港\n股\n通\n新\n经\n济\n指\n数\nA',
 '华\n夏\n恒\n生\nE\nT\nF\n联\n接',
 '易\n方\n达\n中\n概\n互\n联\n5\n0\nE\nT\nF\n联\n接\n人\n民\n币\nA',
 '交\n银\n中\n证\n海\n外\n中\n国\n互\n联\n网\n指\n数',
 '华\n安\n香\n港\n精\n选',
 '中\n欧\n丰\n泓\n沪\n港\n深\nA']

In [71]:
sipmle_name[0]


'华安纳斯达克100指数'

In [115]:
date='20190118'
year = (datetime.datetime.now() - datetime.datetime.strptime(date,'%Y%m%d')).days/365
print(year)

3.0904109589041098


In [116]:
0.98**(1/3.09)

0.9934832338426756

In [100]:
for i in sipmle_name:
    print(i)

华安纳斯达克100指数
博时标普500ETF联接
天弘恒生科技指数A
嘉实港股通新经济指数A
华夏恒生ETF联接
易方达中概互联50ETF联接人民币A
交银中证海外中国互联网指数
华安香港精选
中欧丰泓沪港深A


In [111]:
1-0.7228

0.2772

In [114]:
0.5923-1

-0.40769999999999995

In [117]:
0.9934832338426756-1

-0.006516766157324438

In [123]:
code='006327'
result=[]
for item in fund[code].find({},{'_id':0}):
    result.append(item)
df = pd.DataFrame(result)

In [124]:
df.head()

,净值日期,单位净值,日增长率
0,2019-01-18,1.0000,0.0
1,2019-01-25,1.0001,0.0
2,2019-02-01,1.0003,0.0
3,2019-02-15,1.0007,0.0
4,2019-02-18,1.0007,0.0


In [126]:
money = 10000
total_share =0 
interval = 22
length = len(df)
sum_money=0
for i in range(0,length,interval):
    buy_date_df = df.iloc[i]
    share = money/buy_date_df['单位净值']
    total_share+=share
    sum_money+=money

In [127]:
total_share

285727.29130332154

In [128]:
sum_money

340000

In [134]:
df.iloc[-1]['单位净值']*total_share

280241.32731029775

In [129]:
(df.iloc[-1]['单位净值']*total_share-sum_money)/sum_money*100

-17.576080202853603

In [130]:
from datetime import date
 
# 函数
def xirr(cashflows):
    years = [(ta[0] - cashflows[0][0]).days / 365. for ta in cashflows]
    residual = 1.0
    step = 0.05
    guess = 0.05
    epsilon = 0.0001
    limit = 10000
    while abs(residual) > epsilon and limit > 0:
        limit -= 1
        residual = 0.0
        for i, trans in enumerate(cashflows):
            residual += trans[1] / pow(guess, years[i])
        if abs(residual) > epsilon:
            if residual > 0:
                guess += step
            else:
                guess -= step
                step /= 2.0
    return guess - 1
 
 
# 测试
data = [(datetime.date(2006, 1, 24), -39967), (datetime.date(2008, 2, 6), -19866), (datetime.date(2010, 10, 18), 245706), (datetime.date(2013, 9, 14), 52142)]
xirr(data)

0.42369774132967053

In [133]:
data=[]
for i in range(34):
    data.append((datetime.date(2006, 1, 24)+datetime.timedelta(days=i*30), -10000))

In [135]:
data.append((datetime.date(2006, 1, 24)+datetime.timedelta(days=i*30),280241))

In [136]:
xirr(data)

-0.1373597145080565